## 패스트텍스트(FastText)

Word2Vec 이후에 나온 페이스북에서 개발한 방식

메커니즘 자체는 Word2Vec의 확장

Word2Vec와 FastText와의 가장 큰 차이점은

Word2Vec는 단어를 쪼개질 수 없는 단위로 생각

FastText는 하나의 단어 안에도 여러 단어들이 존재하는 것으로 간주, 서브워드(subword)를 고려하여 학습

### 내부 단어(subword)의 학습

FastText에서는 각 단어는 글자 단위 n-gram의 구성으로 취급

n을 몇으로 결정하는지에 따라서 단어들이 얼마나 분리되는지 결정

예를 들어, n을 3으로 잡은 트라이그램(tri-gram)의 경우, 

apple은 시작과 끝을 의미하는 \<, \>를 도입하여, \<ap, app, ppl, ple, le\> 5개 내부 단어(subword) 토큰을 벡터로 변환

추가적으로 하나를 더 벡터화하는데, 기존 단어에 \<, 와 \>를 붙인 토큰, \<apple\> 를 벡터화

즉 n = 3인 경우, FastText는 단어 apple에 대해서 다음의 6개의 토큰을 벡터화

\<ap, app, ppl, ple, le\>, \<apple\>

실제 사용할 때는 n의 최소값과 최대값으로 범위를 설정 가능, 기본값으로는 각각 3과 6으로 설정

n = 3 ~ 6인 경우 apple의 내부 단어는 다음과 같음

\<ap, app, ppl, ppl, le\>, \<app, appl, pple, ple\>, \<appl, pple\>, ..., \<apple\>

여기서 내부 단어들을 벡터화한다는 의미는 저 단어들에 대해서 Word2Vec을 수행한다는 의미

내부 단어들의 벡터값을 얻었다면, 단어 apple의 벡터값은 저 위 벡터값들의 총 합으로 구성

apple = \<ap + app + ppl + ppl + le\> + \<app + appl + pple + ple\> + \<appl + pple\> + , ..., + \<apple\>

### 모르는 단어(Out Of Vocabulary, OOV)에 대한 대응

FastText의 인공 신경망을 학습한 후에는 데이터 셋의 모든 단어의 각 n-gram에 대해서 워드 임베딩 됨

데이터 셋만 충분한다면 위와 같은 내부 단어(Subword)를 통해 모르는 단어(Out Of Vocabulary, OOV)에 대해서도 다른 단어와의 유사도를 계산 가능

예를 들어, FastText에서 birthplace(출생지)란 단어를 학습하지 않은 상태라 가정하면 

다른 단어에서 birth와 place라는 내부 단어가 있었다면 FastText는 birthplace의 벡터를 얻을 수 있음

이는 모르는 단어에 제대로 대처할 수 없는 Word2Vec, GloVe와는 다른 점

### 단어 집합 내 빈도 수가 적었던 단어(Rare Word)에 대한 대응

Word2Vec의 경우에는 등장 빈도 수가 적은 단어(rare word)에 대해서는 임베딩의 정확도가 높지 않다는 단점이 있음

참고할 수 있는 경우의 수가 적다보니 정확하게 임베딩이 되지 않는 경우 임

FastText의 경우, 만약 단어가 희귀 단어라도, 그 단어의 n-gram이 다른 단어의 n-gram과 겹치는 경우라면, 

Word2Vec과 비교하여 비교적 높은 임베딩 벡터값을 얻을 수 있음

이때문에 FastText가 노이즈가 많은 코퍼스에서 강점을 가짐

모든 훈련 코퍼스에 오타(Typo)나 맞춤법이 틀린 단어가 없으면 이상적이겠지만, 실제 많은 비정형 데이터에는 오타가 섞여있음

오타가 섞인 단어는 당연히 등장 빈도수가 매우 적으므로 일종의 희귀 단어가 됨

Word2Vec에서는 오타가 섞인 단어는 임베딩이 제대로 되지 않지만 FastText는 이에 대해서도 일정 수준의 성능을 보임

### Word2Vec, FastText 비교

In [53]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models import KeyedVectors

In [54]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

In [55]:
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))
print(parse_text[:100])

Here are two reasons companies fail: they only do more of the same, or they only do what's new.
To m


In [56]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)
print(content_text[:100])

Here are two reasons companies fail: they only do more of the same, or they only do what's new.
To m


In [57]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)
print(sent_text[:3])
print(len(sent_text))

["Here are two reasons companies fail: they only do more of the same, or they only do what's new.", 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation.', 'Both are necessary, but it can be too much of a good thing.']
273424


In [58]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

print(normalized_text[:3])
print(len(normalized_text))

['here are two reasons companies fail they only do more of the same or they only do what s new ', 'to me the real real solution to quality growth is figuring out the balance between two activities exploration and exploitation ', 'both are necessary but it can be too much of a good thing ']
273424


In [59]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]
print(result[:3])
print(len(result))

[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation'], ['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']]
273424


In [60]:
# Word2Vec 학습
model_wv = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [61]:
# FastText 학습
model_ft = FastText(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [62]:
# Word2Vec는 모르는 단어에 대해서는 임베딩 벡터가 존재하지 않음
try:
    model_result = model_wv.wv.most_similar("electrofishing")
except Exception as e:
    print(e)

"Key 'electrofishing' not present in vocabulary"


In [63]:
# FastText는 모르는 단어에 대해서 유사한 단어를 계산해서 출력 가능
try:
    model_result = model_ft.wv.most_similar("electrofishing")
    print(model_result)
except Exception as e:
    print(e)

[('electrolyte', 0.8746446967124939), ('electrolux', 0.8699601292610168), ('electro', 0.8579500913619995), ('electroencephalogram', 0.8560504913330078), ('electroshock', 0.8535223007202148), ('electrogram', 0.8337435722351074), ('electrons', 0.8243095874786377), ('electric', 0.8234108686447144), ('electrochemical', 0.822003185749054), ('electron', 0.8217300176620483)]


In [64]:
model_wv.wv.save_word2vec_format('eng_w2v')
model_ft.save('eng_ft')

## 한국어에서의 FastText

한국어의 경우에도 OOV 문제를 해결하기 위해 FastText를 적용하고자 하는 시도들이 있었음

### 음절 단위

음절 단위의 임베딩의 경우에 n=3일때 '자연어처리'라는 단어에 대해 n-gram을 만들어보면 다음과 같음

\<자연, 자연어, 연어처, 어처리, 처리\>

### 자모 단위

더 나아가 자모 단위(초성, 중성, 종성 단위)로 임베딩하는 시도 또한 있었음

음절 단위가 아니라, 자모 단위로 가게 되면 오타나 노이즈 측면에서 더 강한 임베딩을 기대해볼 수 있음

'자연어처리'라는 단어에 대해서 초성, 중성, 종성을 분리하고, 만약, 종성이 존재하지 않는다면 ‘_’라는 토큰을 사용한다고 가정한다면 아래와 같이 분리가 가능

ㅈ ㅏ _ ㅇ ㅕ ㄴ ㅇ ㅓ _ ㅊ ㅓ _ ㄹ ㅣ _

분리된 결과에 대해서 n=3일 때, n-gram을 적용하여, 임베딩을 한다면 다음과 같음

\< ㅈ ㅏ, ㅈ ㅏ _, ㅏ _ ㅇ, ... \>

In [65]:
from jamo import h2j, j2hcj

In [66]:
# 예시 텍스트 데이터
texts = [
    "안녕하세요",
    "반갑습니다",
    "자연어 처리를 배우고 있습니다"
]

In [67]:
# 텍스트 데이터를 자모 단위로 분리하는 함수
def text_to_jamo(text):
    return ' '.join([j for char in text for j in h2j(char)])

In [68]:
# 자모 단위로 분리된 텍스트 데이터 생성
jamo_texts = [text_to_jamo(text) for text in texts]
print(jamo_texts)

['ᄋ ᅡ ᆫ ᄂ ᅧ ᆼ ᄒ ᅡ ᄉ ᅦ ᄋ ᅭ', 'ᄇ ᅡ ᆫ ᄀ ᅡ ᆸ ᄉ ᅳ ᆸ ᄂ ᅵ ᄃ ᅡ', 'ᄌ ᅡ ᄋ ᅧ ᆫ ᄋ ᅥ   ᄎ ᅥ ᄅ ᅵ ᄅ ᅳ ᆯ   ᄇ ᅢ ᄋ ᅮ ᄀ ᅩ   ᄋ ᅵ ᆻ ᄉ ᅳ ᆸ ᄂ ᅵ ᄃ ᅡ']


In [69]:
# 자모 단위로 분리된 텍스트 데이터를 토큰화
tokenized_texts = [text.split() for text in jamo_texts]
print(tokenized_texts)

[['ᄋ', 'ᅡ', 'ᆫ', 'ᄂ', 'ᅧ', 'ᆼ', 'ᄒ', 'ᅡ', 'ᄉ', 'ᅦ', 'ᄋ', 'ᅭ'], ['ᄇ', 'ᅡ', 'ᆫ', 'ᄀ', 'ᅡ', 'ᆸ', 'ᄉ', 'ᅳ', 'ᆸ', 'ᄂ', 'ᅵ', 'ᄃ', 'ᅡ'], ['ᄌ', 'ᅡ', 'ᄋ', 'ᅧ', 'ᆫ', 'ᄋ', 'ᅥ', 'ᄎ', 'ᅥ', 'ᄅ', 'ᅵ', 'ᄅ', 'ᅳ', 'ᆯ', 'ᄇ', 'ᅢ', 'ᄋ', 'ᅮ', 'ᄀ', 'ᅩ', 'ᄋ', 'ᅵ', 'ᆻ', 'ᄉ', 'ᅳ', 'ᆸ', 'ᄂ', 'ᅵ', 'ᄃ', 'ᅡ']]


In [70]:
# FastText 모델 학습
model = FastText(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4, sg=1, epochs=10)

In [71]:
# "안녕하세요"의 자모 단위 벡터 확인
jamo_vector = model.wv[text_to_jamo("안녕하세요").split()]
print(jamo_vector)

[[-0.00222797  0.00187597  0.00436019 ...  0.00302328 -0.00807463
   0.00667974]
 [ 0.00229831 -0.00284881 -0.00224704 ... -0.00838212 -0.00152303
   0.00330817]
 [-0.00710352  0.00064455 -0.00070119 ... -0.00264836 -0.00485959
  -0.00391405]
 ...
 [ 0.00764866 -0.00201815  0.00386967 ...  0.00026743  0.007372
   0.00338729]
 [-0.00222797  0.00187597  0.00436019 ...  0.00302328 -0.00807463
   0.00667974]
 [-0.00122434  0.00388527  0.00217325 ...  0.00559431  0.00189277
   0.00285902]]


In [72]:
# 모델 저장
model.save("fasttext_jamo.model")

In [73]:
# 모델 로드
model = FastText.load("fasttext_jamo.model")